In [2]:
import sys
import os
import re
sys.path.append('../scripts')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import helpers as hp
import pickle as pkl
import itertools as it

from collections import OrderedDict, defaultdict

from sklearn.metrics import f1_score, precision_score, recall_score,\
                            average_precision_score, roc_auc_score,\
                            roc_curve, precision_recall_curve, confusion_matrix,\
                            accuracy_score, classification_report

from IPython.core.interactiveshell import InteractiveShell
from matplotlib import rcParams
from importlib import reload
from model_generator import deep_er_model_generator
pd.options.display.max_colwidth = 1000

rcParams['font.family'] = 'serif'
rcParams['font.serif'] = 'times new roman'

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

/home/paperspace/anaconda3/envs/python-36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load data 

In [3]:
reload(hp)

with open('../data/converted/dblp-scholar-reduced/glove-300.map', 'rb') as f:
    map = pkl.load(f)

data_dir = os.path.join('..','data')
source_dir = os.path.join(data_dir,'split','dblp-scholar-reduced')
data = hp.load_data(source_dir)

datasets = ['train_1', 'val_1', 'test_1', 'train_2', 'val_2', 'test_2']

for data_name in datasets:
    data[data_name] = data[data_name].fillna(0)

doc_freqs_1, doc_freqs_2 = hp.get_document_frequencies('../data/converted/dblp-scholar-reduced/', mapping=map)
nan_idx = map['word2idx']['NaN']

# Generate model

In [4]:
histories = dict(acc=list(), val_acc=list(), loss=list(), val_loss=list())
model, X_train, X_val, X_test, y_train, y_val, y_test = \
deep_er_model_generator(data,
                        embedding_file = '../data/converted/dblp-scholar-reduced/glove-300.matrix.npy',
                        text_columns = ['title', 'authors', 'venue'],
                        numeric_columns = ['year'],
                        text_nan_idx=nan_idx,
                        num_nan_val=0,
                        text_sim_metrics=['cosine', 'inverse_l1'],
                        text_compositions=['average'],
                        numeric_sim_metrics=['min_max_ratio', 'scaled_inverse_lp', 'unscaled_inverse_lp'],
                        dense_nodes=[32, 16, 8],
                        document_frequencies=(doc_freqs_1, doc_freqs_2),
                        idf_smoothing=2,
                        make_isna=True,
                        embedding_trainable=True)

Number of inputs to dense layer: 17


# Train model 

In [30]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=1, batch_size=8192,
                    validation_data=(X_val, y_val),
                    shuffle=True)

histories['acc'].extend(history.history['acc'])
histories['val_acc'].extend(history.history['val_acc'])
histories['loss'].extend(history.history['loss'])
histories['val_loss'].extend(history.history['val_loss'])

Train on 432038 samples, validate on 54004 samples
Epoch 1/1
432038/432038 [==============================] - 20s 45us/step - loss: 1.8025e-04 - acc: 1.0000 - val_loss: 0.0059 - val_acc: 0.9990


# Learning curves

In [32]:
fig, axes = plt.subplots(1, 2, figsize=(16,6))
axes[0].plot(histories['loss'], label='loss')
axes[0].plot(histories['val_loss'], label='val_loss')
axes[0].set_ylim(0, 0.02)
axes[1].plot(histories['acc'], label='acc')
axes[1].plot(histories['val_acc'], label='val_acc')
axes[1].set_ylim(0.99, 1)

for ax in axes:
    ax.legend()
    ax.grid(True)
    ax.set(xlabel='epoch')

# Evaluate model 

In [19]:
y_val_pred = model.predict(X_val, batch_size=8192)[:,1]
y_val_ = data['val_y'].values.squeeze()
print(classification_report(y_val_, y_val_pred >= 0.5))
print(confusion_matrix(y_val_, y_val_pred >= 0.5))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00     53474
          1       0.99      0.91      0.95       530

avg / total       1.00      1.00      1.00     54004

[[53469     5]
 [   47   483]]


In [20]:
mapping = map.copy()
bool_mask = (y_val_ == 1) & ~(y_val_pred >= 0.5)
columns = ['title', 'authors', 'venue']
df_examine = hp.examine_data(data['val_1'], data['val_2'], columns, bool_mask, mapping)
df_examine

id1  \
182                  conf/sigmod/ParkCY95   
1026                 conf/vldb/AgrawalS94   
1454              conf/sigmod/GodfreySE98   
2019                 journals/sigmod/Wu02   
3788         conf/sigmod/RoussopoulosKV95   
5680              conf/sigmod/MaddenSHR02   
7848             conf/sigmod/BettiniWBJ95   
8685             conf/sigmod/AnkerstBKS99   
11286              conf/vldb/ManegoldBK00   
11340           journals/vldb/PapadiasS94   
11699               conf/sigmod/ZhaoDNS98   
13046            conf/sigmod/FaloutsosL95   
13493           conf/vldb/HellersteinNP95   
13926                   conf/vldb/WuYCT95   
15528   conf/sigmod/OConnellISWABCCLPWW96   
16268          conf/sigmod/AggarwalPWYP99   
16589                 conf/vldb/IdrisGC94   
17041            conf/sigmod/FaloutsosL95   
17181          journals/sigmod/CasatiDS00   
18698            conf/sigmod/BarbosaMKL02   
19848            conf/sigmod/FaloutsosL95   
23099           conf/sigmod/BerchtoldBK98   
23432              conf/sigmod/OlstonJW03   
23530   conf/vldb/ShanmugasundaramGTZDN99   
25979              conf/vldb/GardarinST96   
26225             conf/vldb/FaloutsosJS97   
26841               conf/sigmod/GuptaMR95   
27714                  journals/vldb/C94c   
29410            conf/sigmod/FranklinJK96   
29679           conf/sigmod/SaltenisJLL00   
30625          conf/sigmod/ImielinskiVB94   
30812              conf/vldb/KiesslingK02   
31930             conf/sigmod/GravanoGT94   
32799                   conf/vldb/YuSLJ02   
35689           journals/sigmod/MeltonE02   
36199             journals/vldb/ShyyASL96   
36202            conf/sigmod/PoosalaIHS96   
38717                  journals/vldb/C95b   
40241           conf/vldb/HellersteinNP95   
40247               conf/sigmod/HanFHCC94   
42457  journals/vldb/JagadishACLNPPSWWY02   
43003              conf/sigmod/EmbleyJN99   
43614                 conf/sigmod/Carey00   
44367            conf/vldb/SchmidtWKCMB02   
49643                   conf/vldb/CeriM03   
49761                conf/vldb/FangSGMU98   
50787                     conf/vldb/NgH94   

                                                                                                              title  \
182                                                  an effective hash based algorithm for mining association rules   
1026                                                fast algorithms for mining association rules in large databases   
1454                                                                     secure and portable database extensibility   
2019                               small worlds  the dynamics of networks between order and randomness  book review   
3788                                                                                       nearest neighbor queries   
5680                                                          continuously adaptive continuous queries over streams   
7848                                                semantic assumptions and query evaluation in temporal databases   
8685                                                   optics  ordering points to identify the clustering structure   
11286                                     what happens during  join  dissecting cpu and memory optimization effects   
11340                                                     qualitative representation of spatial knowledge in  space   
11699                                      simultaneous optimization and evaluation of multiple dimensional queries   
13046              fastmap   fast algorithm for indexing   and visualization of traditional and multimedia datasets   
13493                                                                 generalized search trees for database systems   
13926    performance study of workfile disk management for concurrent mergesorts in  multiprocessor database system   
15528                                                        multimedia server for massively

# Save model and histories 

In [21]:
pd.DataFrame(histories).to_csv('../data/histories/dblp-scholar-base-1t-num-null-allsim.csv', index=False)
model.save_weights('../data/models/dblp-scholar-base-1t-num-null-allsim')

# Test model 

In [5]:
model.load_weights('../data/models/dblp-scholar-base-1t-num-null-allsim')
y_test_pred = model.predict(X_test, batch_size=8192)[:,1]
y_test_ = data['test_y'].values.squeeze()
print(classification_report(y_test_, y_test_pred >= 0.5, digits=5))
print(confusion_matrix(y_test_, y_test_pred >= 0.5))

             precision    recall  f1-score   support

          0    0.99918   0.99996   0.99957     53467
          1    0.99597   0.91822   0.95551       538

avg / total    0.99915   0.99915   0.99913     54005

[[53465     2]
 [   44   494]]
